# Autotalker Data Preprocessing

- **Creator**: Sebastian Birk (<sebastian.birk@helmholtz-munich.de>).
- **Affiliation:** Helmholtz Munich, Institute of Computational Biology (ICB), Talavera-López Lab
- **Date of Creation:** 01.10.2022
- **Date of Last Modification:** 11.01.2023

## 1. Setup

### 1.1 Import Libraries

In [1]:
import os
from copy import deepcopy

import anndata as ad
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.sparse as sp
import seaborn as sns
import squidpy as sq

### 1.2 Configure Paths and Create Directories

In [2]:
# Define paths
srt_data_folder_path = "../datasets/srt_data" # spatially resolved transcriptomics data
srt_data_bronze_folder_path = f"{srt_data_folder_path}/bronze"
srt_data_silver_folder_path = f"{srt_data_folder_path}/silver"
srt_data_gold_folder_path = f"{srt_data_folder_path}/gold"

# Create required directories
os.makedirs(srt_data_bronze_folder_path, exist_ok=True)
os.makedirs(srt_data_silver_folder_path, exist_ok=True)
os.makedirs(srt_data_gold_folder_path, exist_ok=True)

## 2. Data

### 2.1 seqFISH Mouse Organogenesis Dataset

- **Publication:** Lohoff, T. et al. Highly multiplexed spatially resolved gene expression profiling of mouse organogenesis. bioRxiv 2020.11.20.391896 (2020). [doi:10.1101/2020.11.20.391896](https://doi.org/10.1101/2020.11.20.391896)
- **Data Access:** Preprocessed version from squidpy API and original source https://marionilab.cruk.cam.ac.uk/SpatialMouseAtlas/
- **Summary:**
    - Original source
        - Sagittal tissue sections of 3 8-12 somite stage mouse embryos
        - 19,451 observations (embryo 1), 14,891 observations (embryo 2) and 23,194 observations (embryo 3) on cell-level with cell-type annotations
        - 351 probed genes
        - Dataset is also available with imputed genes based on scRNA-seq

In [3]:
dataset = "seqfish_mouse_organogenesis"
cell_type_key = "celltype_mapped_refined"

#### 2.1.1 Load & Preprocess Raw Data

Preprocessing is done with ```../scripts/seqfish_mouse_organogenesis_data_preparation.R```.

In [4]:
# Read preprocessed data
adata = sc.read_h5ad(f"{srt_data_silver_folder_path}/{dataset}.h5ad")

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '../datasets/srt_data/silver/seqfish_mouse_organogenesis.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [5]:
# Filter low quality cells
adata = adata[adata.obs["celltype_mapped_refined"] != "Low quality"]

In [9]:
# Format adata
adata.obsm["spatial"] = np.column_stack((adata.obs.x, adata.obs.y))
adata.obs.drop(["x", "y"], axis=1, inplace=True)
adata.X = adata.X.astype(np.float32)

/var/folders/wj/ck4tvfm941sdgxng3r0d3lh1k5k8lb/T/ipykernel_96564/2441972308.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata.obsm["spatial"] = np.column_stack((adata.obs.x, adata.obs.y))


In [10]:
# Change cell annotation colors
color_list = ["#d8d4d6", # Allantois
              "#800000", # Anterior somitic tissues
              "#b7efae", # Blood progenitors
              "#805500", # Cardiomyocytes
              "#ffaa00", # Cranial mesoderm
              "#558000", # Definitive endoderm
              "#aaff00", # Dermomyotome
              "#ff0000", # Endothelium
              "#b3ffb3", # Erythroid
              "#008055", # Forebrain/Midbrain/Hindbrain
              "#00ffaa", # Gut tube
              "#005580", # Haematoendothelial progenitors
              "#00aaff", # Intermediate mesoderm
              "#000080", # Lateral plate mesoderm
              # "#0000ff", # Low quality
              "#550080", # Mixed mesenchymal mesoderm
              "#aa00ff", # Neural crest
              "#3997fe", # ExE endoderm
              "#800055", # NMP
              "#ff00aa", # Presomitic mesoderm
              "#80ffff", # Sclerotome^
              "#ff8080", # Spinal cord
              "#000000", # Splanchnic mesoderm
              "#ffff80"] # Surface ectoderm
adata.uns["celltype_mapped_refined_colors"] = color_list

In [11]:
adata_embryo1 = adata[adata.obs.index.str.contains("embryo1")].copy()
adata_embryo2 = adata[adata.obs.index.str.contains("embryo2")].copy()
adata_embryo3 = adata[adata.obs.index.str.contains("embryo3")].copy()

In [7]:
# Store data to disk
adata_embryo1.write(f"{srt_data_gold_folder_path}/{dataset}_embryo1.h5ad")
adata_embryo2.write(f"{srt_data_gold_folder_path}/{dataset}_embryo2.h5ad")
adata_embryo3.write(f"{srt_data_gold_folder_path}/{dataset}_embryo3.h5ad")

Set radius with 5 digits accuracy to get average number of neighbors of 2, 4, 8, 16 and 32 respectively.

In [ ]:
sq.gr.spatial_neighbors(adata_embryo2,
                        coord_type="generic",
                        spatial_key="spatial",
                        radius=0.03154)
avg_edges_per_node = round(adata_embryo2.obsp['spatial_connectivities'].sum(axis=0).mean(), 4)
print(f"Average number of edges per node: {avg_edges_per_node}.")

In [ ]:
embryo2_dict = {}
embryo2_dict["radius_2_avg_neighbors"] = 0.03154
embryo2_dict["radius_4_avg_neighbors"] = 0.03957
embryo2_dict["radius_8_avg_neighbors"] = 0.05194
embryo2_dict["radius_16_avg_neighbors"] = 0.07316
embryo2_dict["radius_32_avg_neighbors"] = 0.10386

In [ ]:
# Create csv files for embryo 2 for DeepLinc method
counts_df = pd.DataFrame(adata_embryo2.X.toarray(), columns=adata_embryo2.var_names)
counts_df.to_csv(f"{srt_data_gold_folder_path}/{dataset}_embryo2_counts.csv", index=False)

coords_df = pd.DataFrame(adata_embryo2.obsm["spatial"], columns=["X", "Y"])
coords_df.to_csv(f"{srt_data_gold_folder_path}/{dataset}_embryo2_coords.csv", index=False)

for n_neighbors in [2, 4, 8, 16, 32]:
    sq.gr.spatial_neighbors(adata_embryo2,
                            coord_type="generic",
                            spatial_key="spatial",
                            radius=embryo2_dict[f"radius_{n_neighbors}_avg_neighbors"])
    adj_df = pd.DataFrame(adata_embryo2.obsp["spatial_connectivities"].toarray())
    adj_df.to_csv(f"{srt_data_gold_folder_path}/{dataset}_embryo2_adj{n_neighbors}.csv", index=False)

cell_types_df = pd.DataFrame(adata_embryo2.obs["celltype_mapped_refined"])
cell_types_df.rename(columns={"celltype_mapped_refined": "Cell_class_name"}, inplace=True)
cell_types_df["Cell_ID"] = np.arange(len(adata_embryo2))
cell_types_df["Cell_class_id"] = cell_types_df["Cell_class_name"].cat.codes
cell_types_df = cell_types_df[["Cell_ID", "Cell_class_id", "Cell_class_name"]]
cell_types_df.to_csv(f"{srt_data_gold_folder_path}/{dataset}_embryo2_cell_types.csv", index=False)

#### 2.1.2 Explore Data

In [ ]:
print(f"Exploring dataset {dataset}_embryo1.")
print(f"Number of nodes (cells): {adata_embryo1.layers['counts'].shape[0]}")
print(f"Number of node features (genes): {adata_embryo1.layers['counts'].shape[1]}")

# Visualize cell-level annotated data in physical space
sq.pl.spatial_scatter(adata_embryo1, color=cell_type_key, shape=None, figsize=(12, 12))

In [ ]:
print(f"Exploring dataset {dataset}_embryo2.")
print(f"Number of nodes (cells): {adata_embryo2.layers['counts'].shape[0]}")
print(f"Number of node features (genes): {adata_embryo2.layers['counts'].shape[1]}")

# Visualize cell-level annotated data in physical space
sq.pl.spatial_scatter(adata_embryo2, color=cell_type_key, shape=None, figsize=(12, 12))

In [ ]:
print(f"Exploring dataset {dataset}_embryo3.")
print(f"Number of nodes (cells): {adata_embryo3.layers['counts'].shape[0]}")
print(f"Number of node features (genes): {adata_embryo3.layers['counts'].shape[1]}")

# Visualize cell-level annotated data in physical space
sq.pl.spatial_scatter(adata_embryo3, color=cell_type_key, shape=None, figsize=(12, 12))

### 2.2 Squidpy Slide-seqV2 Mouse Hippocampus Dataset

- **Publication**: Stickels, R. R. et al. Highly sensitive spatial transcriptomics at near-cellular resolution with Slide-seqV2. Nat. Biotechnol. 39, 313–319 (2021). [doi:10.1038/s41587-020-0739-1](https://doi.org/10.1038/s41587-020-0739-1)
- **Data Access:** Preprocessed version from squidpy API (original source https://singlecell.broadinstitute.org/single_cell/study/SCP815/highly-sensitive-spatial-transcriptomics-at-near-cellular-resolution-with-slide-seqv2)
- **Summary:**
    - Mouse hippocampus puck
    - 41,786 observations on cell-level with cell-type annotations
    - 4,000 probed genes

In [ ]:
dataset = "squidpy_slideseqv2_mouse_hippocampus"
cell_type_key = "cluster"

#### 2.2.1 Load & Preprocess Raw Data

In [ ]:
print(f"Loading and preprocessing dataset '{dataset}'.")

# Retrieve adata from squidpy
adata = sq.datasets.slideseqv2()

# Store raw counts in `layers`
adata.layers["counts"] = adata.X.copy()

# Store data to disk
adata.write(f"{srt_data_gold_folder_path}/{dataset}.h5ad")

#### 2.2.2 Explore Data

In [ ]:
print(f"Exploring dataset {dataset}.")
print(f"Number of nodes (cells): {adata.layers['counts'].shape[0]}")
print(f"Number of node features (genes): {adata.layers['counts'].shape[1]}")

# Visualize cell-level annotated data in physical space
sq.pl.spatial_scatter(adata, color=cell_type_key, shape=None, figsize=(12, 12))

### 2.3 Vizgen MERFISH Mouse Brain Receptor Dataset

- **Publication:** Vizgen Data Release V1.0. May 2021
- **Data Access:** https://info.vizgen.com/mouse-brain-map?submissionGuid=a66ccb7f-87cf-4c55-83b9-5a2b6c0c12b9
    - Slice 1 Replicate 1
        - cell_by_gene_S1R1.csv
        - cell_metadata_S1R1.csv
- **Preprocessing Vignette:** https://squidpy.readthedocs.io/en/latest/external_tutorials/tutorial_vizgen.html
- **Summary:**
    - Coronal section of mouse brain receptor map
    - 78,329 observations on cell-level without annotations
    - 649 probed genes

In [ ]:
dataset = "vizgen_merfish_mouse_brain"
cell_type_key = "leiden"

#### 2.3.1 Load & Preprocess Raw Data

In [ ]:
print(f"Loading and preprocessing dataset '{dataset}'.")

# Read adata from files
merfish_dir = f"{srt_data_bronze_folder_path}/merfish/"
adata = sq.read.vizgen(path=merfish_dir,
                       counts_file="datasets_mouse_brain_map_BrainReceptorShowcase_Slice1_Replicate1_cell_by_gene_S1R1.csv",
                       meta_file="datasets_mouse_brain_map_BrainReceptorShowcase_Slice1_Replicate1_cell_metadata_S1R1.csv")
# Preprocess as per squidpy vignette
adata.var_names.unique()
adata.var["mt"] = adata.var_names.str.startswith("mt-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=(50, 100, 200, 300), inplace=True)
fig, axs = plt.subplots(1, 4, figsize=(15, 4))
sns.histplot(adata.obs["total_counts"],
             kde=False,
             ax=axs[0])
sns.histplot(adata.obs["total_counts"][adata.obs["total_counts"] < 10000],
             kde=False,
             bins=40,
             ax=axs[1])
sns.histplot(adata.obs["n_genes_by_counts"],
             kde=False,
             bins=60,
             ax=axs[2])
sns.histplot(adata.obs["n_genes_by_counts"][adata.obs["n_genes_by_counts"] < 4000],
             kde=False,
             bins=60,
             ax=axs[3])
sc.pp.filter_cells(adata, min_counts=10)
sc.pp.filter_genes(adata, min_cells=10)

# Store raw counts in `layers`
adata.layers["counts"] = adata.X.copy()

# Store data to disk
adata.write(f"{srt_data_silver_folder_path}/{dataset}.h5ad")

# Determine cell annotation as per squidpy vignette
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=4000)
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata)
sc.pl.umap(adata,
           color=["total_counts", "n_genes_by_counts", "leiden"])

# Store data to disk
adata.write(f"{srt_data_gold_folder_path}/{dataset}.h5ad")

#### 2.3.2 Explore Data

In [ ]:
print(f"Exploring dataset {dataset}.")
print(f"Number of nodes (cells): {adata.layers['counts'].shape[0]}")
print(f"Number of node features (genes): {adata.layers['counts'].shape[1]}")

# Visualize cell-level annotated data in physical space
sq.pl.spatial_scatter(adata, color=cell_type_key, shape=None, figsize=(12, 12))

### 2.4 Vizgen MERFISH Mouse Liver Dataset

- **Publication**: Vizgen MERFISH Mouse Liver Map January 2022
- **Data Access:** https://info.vizgen.com/mouse-liver-access
    - Animal 1 replicate 1
        - cell_by_gene.csv
        - cell_metadata.csv
- **Preprocessing Vignette:** https://squidpy.readthedocs.io/en/latest/external_tutorials/tutorial_vizgen_mouse_liver.html
- **Summary:**
    - Liver tissue section of mouse liver map
    - 367,335 observations on cell-level without annotations
    - 385 probed genes

In [ ]:
dataset = "vizgen_merfish_mouse_liver"
cell_type_key = "Cell_Type"

#### 2.4.1 Load & Preprocess Raw Data

In [ ]:
print(f"Loading and preprocessing dataset '{dataset}'.")
    
# Read adata from files
merfish_dir = f"{srt_data_bronze_folder_path}/merfish/"
adata = sq.read.vizgen(merfish_dir,
                       counts_file="Liver1Slice1_cell_by_gene.csv",
                       meta_file="Liver1Slice1_cell_metadata.csv")

# Preprocess as per squidpy vignette
adata.var_names_make_unique()
adata.var["mt"] = adata.var_names.str.startswith("mt-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"],percent_top=(50, 100, 200, 300), inplace=True)
sc.pp.filter_cells(adata, min_counts=50)
sc.pp.filter_genes(adata, min_cells=10)

# Store raw counts in `layers`
adata.layers["counts"] = adata.X.copy()

# Store data to disk
adata.write(f"{srt_data_silver_folder_path}/{dataset}.h5ad")

# Determine cell annotation as per squidpy vignette
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver="arpack")
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=20)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=1.5)

sc.set_figure_params(figsize=(10,10))
sc.pl.umap(adata, color=["leiden"], size=5)

gene_panel = "https://static-content.springer.com/esm/art%3A10.1038%2Fs41421-021-00266-1/MediaObjects/41421_2021_266_MOESM1_ESM.xlsx"
df_ref_panel_ini = pd.read_excel(gene_panel, index_col=0)
df_ref_panel = df_ref_panel_ini.iloc[1:,:1]
df_ref_panel.index.name = None
df_ref_panel.columns = ["Function"]

# Assign marker gene metadata using reference dataset
marker_genes = df_ref_panel[df_ref_panel["Function"].str.contains("marker")].index.tolist()

meta_gene = deepcopy(adata.var)
common_marker_genes = list(set(meta_gene.index.tolist()).intersection(marker_genes))
meta_gene.loc[common_marker_genes, "Markers"] = df_ref_panel.loc[common_marker_genes, "Function"]
meta_gene["Markers"] = meta_gene["Markers"].apply(lambda x: "N.A." if "marker" not in str(x) else x)
meta_gene["Markers"].value_counts()

ser_counts = adata.obs["leiden"].value_counts()
ser_counts.name = "cell counts"
meta_leiden = pd.DataFrame(ser_counts)

cat_name = "leiden"
sig_leiden = pd.DataFrame(columns=adata.var_names, index=adata.obs[cat_name].cat.categories)
for clust in adata.obs[cat_name].cat.categories:
    sig_leiden.loc[clust] = adata[adata.obs[cat_name].isin([clust]),:].X.mean(0)
sig_leiden = sig_leiden.transpose()
leiden_clusters = ["Leiden-" + str(x) for x in sig_leiden.columns.tolist()]
sig_leiden.columns = leiden_clusters
meta_leiden.index = sig_leiden.columns.tolist()
meta_leiden["leiden"] = pd.Series(meta_leiden.index.tolist(), index=meta_leiden.index.tolist())

meta_gene = pd.DataFrame(index=sig_leiden.index.tolist())
meta_gene["info"] = pd.Series("", index=meta_gene.index.tolist())
meta_gene["Markers"] = pd.Series("N.A.", index=sig_leiden.index.tolist())
meta_gene.loc[common_marker_genes, "Markers"] = df_ref_panel.loc[common_marker_genes, "Function"]

meta_leiden["Cell_Type"] = pd.Series("N.A.", index=meta_leiden.index.tolist())
num_top_genes = 30
for inst_cluster in sig_leiden.columns.tolist():
    top_genes = sig_leiden[inst_cluster].sort_values(ascending=False).index.tolist()[:num_top_genes]

    inst_ser = meta_gene.loc[top_genes, "Markers"]
    inst_ser = inst_ser[inst_ser != "N.A."]
    ser_counts = inst_ser.value_counts()
  
    max_count = ser_counts.max()
  
    max_cat = "_".join(sorted(ser_counts[ser_counts == max_count].index.tolist()))
    max_cat = max_cat.replace(" marker", "").replace(" ", "-")
  
    print(inst_cluster, max_cat)
    meta_leiden.loc[inst_cluster, "Cell_Type"] = max_cat

# Rename clusters
meta_leiden["name"] = meta_leiden.apply(lambda x: x["Cell_Type"] + "_" + x["leiden"] , axis=1)
leiden_names = meta_leiden["name"].values.tolist()
meta_leiden.index = leiden_names

# Transfer cell type labels to single cells
leiden_to_cell_type = deepcopy(meta_leiden)
leiden_to_cell_type.set_index("leiden", inplace=True)
leiden_to_cell_type.index.name = None

adata.obs["Cell_Type"] = adata.obs["leiden"].apply(lambda x: leiden_to_cell_type.loc["Leiden-" + str(x), "Cell_Type"])
adata.obs["Cluster"] = adata.obs["leiden"].apply(lambda x: leiden_to_cell_type.loc["Leiden-" + str(x), "name"])

# Store data to disk
adata.write(f"{srt_data_gold_folder_path}/{dataset}.h5ad")

#### 2.4.2 Explore Data

In [ ]:
print(f"Exploring dataset {dataset}.")
print(f"Number of nodes (cells): {adata.layers['counts'].shape[0]}")
print(f"Number of node features (genes): {adata.layers['counts'].shape[1]}")

# Visualize cell-level annotated data in physical space
sq.pl.spatial_scatter(adata, color=cell_type_key, shape=None, figsize=(12, 12))

### 2.5 Vizgen MERFISH Human Liver Cancer Dataset

- **Publication:** Vizgen MERFISH FFPE Human Immuno-oncology Data Set,  May 2022
- **Data Access:** https://info.vizgen.com/ffpe-showcase?submissionGuid=bbe0d6ca-92ad-4257-bdcd-cbc6dee8219b
    - Liver cancer 1
        - cell_by_gene.csv
        - cell_metadata.csv
- **Summary:**
    - FFPE tissue section of human liver cancer
    - 480,592 observations on cell-level without annotations
    - 500 probed genes

In [ ]:
dataset = "vizgen_merfish_human_liver_cancer"
cell_type_key = "leiden"

#### 2.5.1 Load & Preprocess Raw Data

In [ ]:
print(f"Loading and preprocessing dataset '{dataset}'.")
    
# Read counts and remove blank genes
vizgen_dir = f"{srt_data_bronze_folder_path}/merfish/"
adata = ad.read_text(vizgen_dir + "HumanLiverCancerPatient1_cell_by_gene.csv" ,delimiter=",", first_column_names=True)
blank_genes = np.array(["Blank" in v for v in adata.var_names])
adata.obsm["blank_genes"] = pd.DataFrame(adata[:, blank_genes].X.copy(), columns=adata.var_names[blank_genes], index=adata.obs_names)
adata = adata[:, ~blank_genes].copy()
adata.X = sp.csr_matrix(adata.X)

# Read metadata and align index for join
metadata = pd.read_csv(vizgen_dir + "HumanLiverCancerPatient1_cell_metadata.csv", header=0, index_col=0)
metadata.sort_index(inplace=True)
metadata.index = metadata.index.astype("str")

# Join counts with obs and store spatial coordinates
adata.obs = pd.merge(adata.obs, metadata, how="left", left_index=True, right_index=True)
adata.obsm["spatial"] = adata.obs[["center_x", "center_y"]].values
adata.obs.drop(columns=["center_x", "center_y"], inplace=True)

# Preprocess
adata.var_names_make_unique()
adata.var["mt"] = adata.var_names.str.startswith("mt-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=(50, 100, 200, 300), inplace=True)
sc.pp.filter_cells(adata, min_counts=50)
sc.pp.filter_genes(adata, min_cells=10)

# Store raw counts in `layers`
adata.layers["counts"] = adata.X.copy()

# Store data to disk
adata.write(f"{srt_data_silver_folder_path}/{dataset}.h5ad")

# Determine cell annotation
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata)
sc.pl.umap(adata,
           color=["total_counts", "n_genes_by_counts", "leiden"])

# Store data to disk
adata.write(f"{srt_data_gold_folder_path}/{dataset}.h5ad")

#### 2.5.2 Explore Data

In [ ]:
print(f"Exploring dataset {dataset}.")
print(f"Number of nodes (cells): {adata.layers['counts'].shape[0]}")
print(f"Number of node features (genes): {adata.layers['counts'].shape[1]}")

# Visualize cell-level annotated data in physical space
sq.pl.spatial_scatter(adata, color=cell_type_key, shape=None, figsize=(12, 12))

### 2.6 10x Genomics Xenium Human Breast Cancer Tumor Microenvironment Dataset

- **Publication:** Janesick, A. et al. High resolution mapping of the breast cancer tumor microenvironment using integrated single cell, spatial and in situ analysis of FFPE tissue. bioRxiv 2022.10.06.510405 (2022). [doi:10.1101/2022.10.06.510405](https://doi.org/10.1101/2022.10.06.510405)
- **Data Access:** https://www.10xgenomics.com/products/xenium-in-situ/preview-dataset-human-breast
    - Feature-cell matrix (HDF5)
    - Cell summary file (CSV) -> unpack with ```gzip -d Xenium_FFPE_Human_Breast_Cancer_Rep1_cells.csv.gz```
- **Preprocessing Vignette:** https://squidpy.readthedocs.io/en/latest/external_tutorials/tutorial_xenium.html
- **Summary**:
    - FFPE tissue section of human breast cancer
    - 164,079 observations on cell-level without annotations
    - 313 probed genes

In [ ]:
dataset = "10x_xenium_human_breast_cancer"
cell_type_key = "leiden"

#### 2.6.1 Load & Preprocess Raw Data

In [ ]:
print(f"Loading and preprocessing dataset '{dataset}'.")

# Read adata from file and add metadata
xenium_dir = f"{srt_data_bronze_folder_path}/xenium/"
adata = sc.read_10x_h5(filename=xenium_dir + "Xenium_FFPE_Human_Breast_Cancer_Rep1_cell_feature_matrix.h5")
metadata = pd.read_csv(xenium_dir + "Xenium_FFPE_Human_Breast_Cancer_Rep1_cells.csv")
metadata.set_index(adata.obs_names, inplace=True)
adata.obs = metadata.copy()

# Preprocess as per squidpy vignette
adata.var_names_make_unique()
adata.var["mt"] = adata.var_names.str.startswith("mt-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"],percent_top=(50, 100, 200, 300), inplace=True)
sc.pp.filter_cells(adata, min_counts=10)
sc.pp.filter_genes(adata, min_cells=5)

# Store counts in `layers` and spatial coords in `obsm`
adata.layers["counts"] = adata.X.copy()
adata.obsm["spatial"] = adata.obs[["x_centroid", "y_centroid"]].copy().to_numpy()

# Store data to disk
adata.write(f"{srt_data_silver_folder_path}/{dataset}.h5ad")

# Determine cell annotation as per squidpy vignette
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata)

# Store data to disk
adata.write(f"{srt_data_gold_folder_path}/{dataset}.h5ad")

#### 2.6.2 Explore Data

In [ ]:
print(f"Exploring dataset {dataset}.")
print(f"Number of nodes (cells): {adata.layers['counts'].shape[0]}")
print(f"Number of node features (genes): {adata.layers['counts'].shape[1]}")

# Visualize cell-level annotated data in physical space
sq.pl.spatial_scatter(adata, color=cell_type_key, shape=None, figsize=(12, 12))

### 2.7 STARmap PLUS Mouse Central Nervous System Dataset

- **Publication:** Shi, H. et al. Spatial Atlas of the Mouse Central Nervous System at Molecular Resolution. bioRxiv 2022.06.20.496914 (2022). [doi:10.1101/2022.06.20.496914](https://doi.org/10.1101/2022.06.20.496914)
- **Data Access:** https://singlecell.broadinstitute.org/single_cell/study/SCP1830
    - Sagittal 1 (Slice 3)
        - sagittal1_spatial.csv
        - sagittal1raw_expression_pd.csv
- **Summary:**
    - Sagittal section of mouse brain
    - 91,246 observations on cell-level with cell-type annotations
    - 1022 genes

In [ ]:
dataset = "starmap_plus_mouse_cns"
cell_type_key = "Maintype_Symbol"

#### 2.7.1 Load & Preprocess Raw Data

In [ ]:
print(f"Loading and preprocessing dataset '{dataset}'.")

# Read counts and metadata and change format
starmap_plus_dir = f"{srt_data_bronze_folder_path}/starmap_plus/"
metadata = pd.read_csv(starmap_plus_dir + "sagittal1_spatial.csv", skiprows=[1])
counts = pd.read_csv(starmap_plus_dir + "sagittal1raw_expression_pd.csv")
counts.set_index("GENE", inplace=True)
metadata.set_index("NAME", inplace=True)
counts = counts.T

# Merge counts with metadata and store counts in `layers` and spatial coords in `obsm`
adata = ad.AnnData(counts, dtype=np.float32)
adata.obs = pd.merge(adata.obs, metadata, how="left", left_index=True, right_index=True)
adata.obsm["spatial"] = adata.obs[["X", "Y"]].values / 1000 # smaller scale for plotting with squidpy
adata.obs.drop(columns=["X", "Y", "Z"], inplace=True)
adata.layers["counts"] = adata.X.copy()

# Store data to disk
adata.write(f"{srt_data_gold_folder_path}/{dataset}.h5ad")

#### 2.7.2 Explore Data

In [ ]:
print(f"Exploring dataset {dataset}.")
print(f"Number of nodes (cells): {adata.layers['counts'].shape[0]}")
print(f"Number of node features (genes): {adata.layers['counts'].shape[1]}")

# Visualize cell-level annotated data in physical space
sq.pl.spatial_scatter(adata, color=cell_type_key, shape=None, figsize=(12, 12))

### 2.8 EEL FISH Mouse Brain Dataset

- **Publication:** Borm, L. E. et al. Scalable in situ single-cell profiling by electrophoretic capture of mRNA using EEL FISH. Nat. Biotechnol. (2022) [doi:10.1038/s41587-022-01455-3](https://doi.org/10.1038/s41587-022-01455-3)
- **Data Accress:** https://figshare.com/articles/dataset/EEL_Mouse_440_genes_single_cell_data/20310771
    - LBEXP20210718_EEL_Mouse_448_2_20220512.h5ad
- **Summary:**
    - Sagittal mouse brain section
    - 127,591 observations on cell-level with cluster annotations 
    - 440 genes

In [ ]:
dataset = "eel_fish_mouse_brain"
cell_type_key = "Clusters"

#### 2.8.1 Load & Preprocess Raw Data

In [ ]:
print(f"Loading and preprocessing dataset '{dataset}'.")
    
# Read adata from file
eel_fish_dir = f"{srt_data_bronze_folder_path}/eel_fish/"
adata = sc.read_h5ad(eel_fish_dir + "LBEXP20210718_EEL_Mouse_448_2_20220512.h5ad")

# Store counts in `layers` and spatial coords in `obsm`
adata.obsm["spatial"] = adata.obs[["X", "Y"]].values # smaller scale for plotting with squidpy
adata.obs.drop(columns=["X", "Y", "X_um", "Y_um"], inplace=True)
adata.layers["counts"] = adata.X.copy()

# Store data to disk
adata.write(f"{srt_data_gold_folder_path}/{dataset}.h5ad")

#### 2.8.2 Explore Data

In [ ]:
print(f"Exploring dataset {dataset}.")
print(f"Number of nodes (cells): {adata.layers['counts'].shape[0]}")
print(f"Number of node features (genes): {adata.layers['counts'].shape[1]}")

# Visualize cell-level annotated data in physical space
sq.pl.spatial_scatter(adata, color=cell_type_key, shape=None, figsize=(12, 12))